In [8]:
import cv2
import mediapipe as mp
import numpy as np
import csv
import os
from datetime import datetime

# Initialize MediaPipe solutions
mp_face_mesh = mp.solutions.face_mesh
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

def get_avg_color(region):
    avg_color_per_row = np.average(region, axis=0)
    avg_color = np.average(avg_color_per_row, axis=0)
    return avg_color

def bgr_to_hex(bgr):
    r, g, b = int(bgr[2]), int(bgr[1]), int(bgr[0])
    return "#{:02x}{:02x}{:02x}".format(r, g, b).upper()

def sample_feature_color(frame, landmarks, idx, region_size=6):
    h, w, _ = frame.shape
    x = int(landmarks.landmark[idx].x * w)
    y = int(landmarks.landmark[idx].y * h)
    patch = frame[y-region_size:y+region_size, x-region_size:x+region_size]
    if patch.size == 0:
        return None, (x, y)
    avg_bgr = get_avg_color(patch)
    return avg_bgr, (x, y)

def initialize_csv(filename):
    if not os.path.exists(filename):
        with open(filename, 'w', newline='') as csvfile:
            csv_writer = csv.writer(csvfile)
            csv_writer.writerow(['Timestamp', 'Eye_R', 'Eye_G', 'Eye_B', 'Eye_Hex', 
                               'Lips_R', 'Lips_G', 'Lips_B', 'Lips_Hex',
                               'Cheek_R', 'Cheek_G', 'Cheek_B', 'Cheek_Hex',
                               'Hair_R', 'Hair_G', 'Hair_B', 'Hair_Hex'])
    return filename

def append_to_csv(filename, feature_colors):
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    row_data = [timestamp]
    
    for feature in ['Eye', 'Lips', 'Cheek', 'Hair']:
        if feature in feature_colors and feature_colors[feature] is not None:
            color_bgr = feature_colors[feature]
            r, g, b = int(color_bgr[2]), int(color_bgr[1]), int(color_bgr[0])
            hex_color = bgr_to_hex(color_bgr)
            row_data.extend([r, g, b, hex_color])
        else:
            row_data.extend([None, None, None, None])
    
    with open(filename, 'a', newline='') as csvfile:
        csv_writer = csv.writer(csvfile)
        csv_writer.writerow(row_data)

def start_color_analysis():
    csv_filename = "av-features.csv"
    initialize_csv(csv_filename)
    
    cap = cv2.VideoCapture(0)
    
    # Custom drawing specifications with blue color for mesh
    blue_mesh_spec = mp_drawing.DrawingSpec(color=(250, 17, 219), thickness=1, circle_radius=1)
    
    # MediaPipe FaceMesh configuration
    with mp_face_mesh.FaceMesh(
        max_num_faces=1,
        refine_landmarks=True,
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5
    ) as face_mesh:
        
        recording = False
        show_mesh = True  # Toggle for showing/hiding the facial mesh
        feature_colors = {}
        
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
                
            # Convert to RGB for MediaPipe
            image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            # Process the image
            results = face_mesh.process(image_rgb)
            
            # Make a copy of the frame for visualization
            display_frame = frame.copy()
            
            if results.multi_face_landmarks:
                face_landmarks = results.multi_face_landmarks[0]
                
                # Draw the facial mesh if enabled
                if show_mesh:
                    # Draw tessellation with custom blue color
                    mp_drawing.draw_landmarks(
                        image=display_frame,
                        landmark_list=face_landmarks,
                        connections=mp_face_mesh.FACEMESH_TESSELATION,
                        landmark_drawing_spec=None,
                        connection_drawing_spec=blue_mesh_spec
                    )
                
                features = {
                    'Eye': 468,   # Left iris center
                    'Lips': 13,   # Lower lip center
                    'Cheek': 205, # Left cheek
                    'Hair': 295   # Eyebrow, in place for hair color
                }
                
                # Reset feature_colors for this frame
                feature_colors = {}
                
                y_offset = 20
                for i, (feature, idx) in enumerate(features.items()):
                    color_bgr, (x, y) = sample_feature_color(frame, face_landmarks, idx)
                    if color_bgr is not None:
                        feature_colors[feature] = color_bgr
                        r, g, b = int(color_bgr[2]), int(color_bgr[1]), int(color_bgr[0])
                        hex_color = bgr_to_hex(color_bgr)
                        
                        # Text display
                        text = f"{feature}: RGB({r},{g},{b}) | {hex_color}"
                        cv2.putText(display_frame, text, (10, 30 + y_offset * i), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
                        
                        # Color swatch next to face
                        swatch_x = x + 10
                        swatch_y = y - 10
                        cv2.rectangle(display_frame, (swatch_x, swatch_y), (swatch_x + 40, swatch_y + 20), (b, g, r), -1)
                        cv2.rectangle(display_frame, (swatch_x, swatch_y), (swatch_x + 40, swatch_y + 20), (0, 0, 0), 1)
                        
                        # Highlight feature point with bright red (to stand out against blue mesh)
                        cv2.circle(display_frame, (x, y), 3, (255, 255, 255), -1)
            
            # Display recording status
            if recording:
                cv2.putText(display_frame, "Recording: ON (Press 'r' to stop)", (10, display_frame.shape[0] - 20), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)
            else:
                cv2.putText(display_frame, "Recording: OFF (Press 'r' to start)", (10, display_frame.shape[0] - 20), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
                
            # Display CSV status and controls
            cv2.putText(display_frame, f"CSV: {csv_filename}", (10, display_frame.shape[0] - 50), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1)
            cv2.putText(display_frame, "Press 'm' to toggle mesh", (10, display_frame.shape[0] - 80), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1)
                       
            # Show current frame    
            cv2.imshow('KoreAI Analyzer – Feature Colors', display_frame)
            
            # Handle keyboard input
            key = cv2.waitKey(5) & 0xFF
            if key == 27:  # ESC key
                break
            elif key == ord('r'):  # Toggle recording
                recording = not recording
                if recording:
                    print(f"Recording started. Data will be saved to {csv_filename}")
                else:
                    print(f"Recording stopped. Data saved to {csv_filename}")
            elif key == ord('s') and len(feature_colors) > 0:  # Manual save current frame
                append_to_csv(csv_filename, feature_colors)
                print(f"Snapshot saved to {csv_filename}")
            elif key == ord('m'):  # Toggle mesh visualization
                show_mesh = not show_mesh
                print(f"Face mesh visualization: {'ON' if show_mesh else 'OFF'}")
            
            # If recording and we have detected features, append to CSV
            if recording and results.multi_face_landmarks and len(feature_colors) > 0:
                append_to_csv(csv_filename, feature_colors)
                
    cap.release()
    cv2.destroyAllWindows()
    print(f"Analysis complete. Data saved to {csv_filename}")

if __name__ == "__main__":
    start_color_analysis()

2025-04-06 09:21:49.557 python[12387:228609] WARNING: AVCaptureDeviceTypeExternal is deprecated for Continuity Cameras. Please use AVCaptureDeviceTypeContinuityCamera and add NSCameraUseContinuityCameraDeviceType to your Info.plist.
I0000 00:00:1743956510.860650  228609 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M3 Pro
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1743956510.871788  246181 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1743956510.880907  246189 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1743956510.901252  246181 landmark_projection_calculator.cc:186] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.


KeyboardInterrupt: 

In [1]:
import pandas as pd

def csv_to_df(file_name):
    df = pd.read_csv(file_name)[['Eye_R', 'Eye_G', 'Eye_B', 
                                'Lips_R', 'Lips_G', 'Lips_B',
                                'Cheek_R', 'Cheek_G', 'Cheek_B',
                                'Hair_R', 'Hair_G', 'Hair_B']]
    avg = df.mean(axis=0)
    transpose = pd.DataFrame(avg).transpose()
    return transpose

vk = csv_to_df('vk-features.csv')
dd = csv_to_df('dd-features.csv')
sp = csv_to_df('sp-features.csv')
av = csv_to_df('av-features.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'vk-features.csv'

In [14]:
# Undertone

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Load the data
training = pd.read_csv('../csv/training_celebs.csv')[['undertone', 'Eye_R', 'Eye_G', 'Eye_B', 
                                           'Lips_R', 'Lips_G', 'Lips_B', 'Cheek_R', 'Cheek_G', 
                                           'Cheek_B', 'Hair_R', 'Hair_G', 'Hair_B']]

# Prepare the features and target variable
X = training.drop(columns=['undertone'])
y = training['undertone']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# List of numerical columns (features that are already numeric)
num_columns = ['Eye_R', 'Eye_G', 'Eye_B', 'Lips_R', 'Lips_G', 'Lips_B', 
               'Cheek_R', 'Cheek_G', 'Cheek_B', 'Hair_R', 'Hair_G', 'Hair_B']

from sklearn.linear_model import LogisticRegression

pipeline = Pipeline(steps=[
    ('classifier', LogisticRegression())
])

# Train the model
pipeline.fit(X_train, y_train)

# Make predictions
def predict_undertone(features):
    pred = pipeline.predict(features)
    return pred

import joblib

# Save the pipeline to a file
joblib.dump(pipeline, '../models/undertone_classifier3.pkl')

# predict_undertone(av)

# Evaluate the model
y_pred = pipeline.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))  

Accuracy: 0.4


/Users/snigdhapodugu/miniforge3/envs/dsc80/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [11]:
# Season

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Load the data
training = pd.read_csv('../csv/training_celebs.csv')[['season', 'Eye_R', 'Eye_G', 'Eye_B', 
                                           'Lips_R', 'Lips_G', 'Lips_B', 'Cheek_R', 'Cheek_G', 
                                           'Cheek_B', 'Hair_R', 'Hair_G', 'Hair_B']]

# Prepare the features and target variable
X = training.drop(columns=['season'])
y = training['season']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# List of numerical columns (features that are already numeric)
num_columns = ['Eye_R', 'Eye_G', 'Eye_B', 'Lips_R', 'Lips_G', 'Lips_B', 
               'Cheek_R', 'Cheek_G', 'Cheek_B', 'Hair_R', 'Hair_G', 'Hair_B']

from sklearn.neural_network import MLPClassifier

from sklearn.linear_model import LogisticRegression

pipeline = Pipeline(steps=[
    ('classifier', LogisticRegression())
])

# Train the model
pipeline.fit(X_train, y_train)

# Make predictions
def predict_season(features):
    pred = pipeline.predict(features)
    return pred

import joblib

# Save the pipeline to a file
joblib.dump(pipeline, '../models/season_classifier3.pkl')

# Evaluate the model
y_pred = pipeline.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))

Accuracy: 0.2


/Users/snigdhapodugu/miniforge3/envs/dsc80/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [3]:
import numpy as np
import colorsys
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
import joblib

class ClothingColorGenerator:
    def __init__(self):
        # Initialize models for hue, saturation, and lightness predictions
        self.hue_model = RandomForestRegressor(n_estimators=100, random_state=42)
        self.sat_model = RandomForestRegressor(n_estimators=100, random_state=42)
        self.light_model = RandomForestRegressor(n_estimators=100, random_state=42)
        
        # Initialize scaler for input features
        self.scaler = StandardScaler()
        
        # Train the models with synthetic data if no pre-trained models exist
        self._train_with_synthetic_data()
        
    def _train_with_synthetic_data(self):
        """Train models with synthetic data representing color harmony principles"""
        # Generate synthetic data based on color theory and seasonal preferences
        # For real applications, this would be replaced with a dataset of expert-curated colors
        
        # Generate samples: skin colors (RGB) x seasons
        np.random.seed(42)
        n_samples = 5000
        
        # Random skin tones across a spectrum
        skin_r = np.random.uniform(0.2, 0.9, n_samples)
        skin_g = np.random.uniform(0.2, 0.8, n_samples)
        skin_b = np.random.uniform(0.1, 0.7, n_samples)
        
        # Convert to HSL for feature engineering
        skin_hsl = np.array([colorsys.rgb_to_hls(r, g, b) for r, g, b in zip(skin_r, skin_g, skin_b)])
        skin_h, skin_l, skin_s = skin_hsl[:, 0], skin_hsl[:, 1], skin_hsl[:, 2]
        
        # Season encoding (one-hot)
        seasons = np.random.choice(['summer', 'autumn', 'winter', 'spring'], n_samples)
        season_summer = (seasons == 'summer').astype(int)
        season_autumn = (seasons == 'autumn').astype(int)
        season_winter = (seasons == 'winter').astype(int)
        season_spring = (seasons == 'spring').astype(int)
        
        # Feature matrix
        X = np.column_stack([
            skin_r, skin_g, skin_b, 
            skin_h, skin_l, skin_s,
            season_summer, season_autumn, season_winter, season_spring
        ])
        
        # Scale features
        self.scaler.fit(X)
        X_scaled = self.scaler.transform(X)
        
        # Generate target colors based on color theory
        # For each skin tone and season, create multiple harmonious colors
        
        # Helper functions to generate color harmonies with seasonal adjustments
        def generate_hue_targets(h, season):
            """Generate harmonious hues with seasonal bias"""
            h_deg = h * 360
            
            # Base harmonies (analogous, complementary, triadic)
            harmonies = [
                (h_deg + 30) % 360,  # analogous +30
                (h_deg - 30) % 360,  # analogous -30
                (h_deg + 180) % 360,  # complementary
                (h_deg + 120) % 360,  # triadic +120
                (h_deg - 120) % 360,  # triadic -120
            ]
            
            # Season-specific hue shifts
            season_shifts = {
                'summer': [-10, 0, 10, 20, 30],  # cooler colors
                'autumn': [-30, -20, -10, 0, 10],  # warmer, earthier tones
                'winter': [0, 10, 20, 30, 40],  # high contrast
                'spring': [-20, -10, 0, 10, 20]   # bright, clear colors
            }
            
            # Apply seasonal shifts to each harmony
            results = []
            for harmony in harmonies:
                for shift in season_shifts[season]:
                    results.append((harmony + shift) % 360)
            
            return np.array(results) / 360  # Normalize back to [0, 1]
        
        def generate_saturation_targets(s, season):
            """Generate saturation values with seasonal adjustments"""
            base = s * 0.8  # Start with a relation to skin saturation
            
            # Season-specific saturation adjustments
            modifiers = {
                'summer': np.linspace(0.6, 0.8, 25),  # softer, muted
                'autumn': np.linspace(0.7, 0.9, 25),  # rich, warm
                'winter': np.linspace(0.8, 1.0, 25),  # clear, bright
                'spring': np.linspace(0.7, 0.95, 25)  # clear, bright
            }
            
            return np.clip(base + modifiers[season], 0.1, 1.0)
        
        def generate_lightness_targets(l, season):
            """Generate lightness values with seasonal adjustments"""
            base = max(0.3, min(0.7, l))  # Keep within reasonable bounds
            
            # Season-specific lightness adjustments
            modifiers = {
                'summer': np.linspace(0.5, 0.8, 25),  # light to medium
                'autumn': np.linspace(0.4, 0.7, 25),  # medium
                'winter': np.linspace(0.3, 0.9, 25),  # high contrast
                'spring': np.linspace(0.6, 0.9, 25)   # light to medium-light
            }
            
            return np.clip(base + modifiers[season] - 0.5, 0.1, 0.9)
        
        # Generate target values for each sample
        y_hue = []
        y_sat = []
        y_light = []
        
        for i in range(n_samples):
            h_targets = generate_hue_targets(skin_h[i], seasons[i])
            s_targets = generate_saturation_targets(skin_s[i], seasons[i])
            l_targets = generate_lightness_targets(skin_l[i], seasons[i])
            
            # Expand X by repeating each row for each target color
            if i == 0:
                X_expanded = np.repeat([X_scaled[i]], len(h_targets), axis=0)
            else:
                X_expanded = np.vstack((X_expanded, np.repeat([X_scaled[i]], len(h_targets), axis=0)))
            
            y_hue.extend(h_targets)
            y_sat.extend(s_targets)
            y_light.extend(l_targets)
        
        # Train models
        self.hue_model.fit(X_expanded, y_hue)
        self.sat_model.fit(X_expanded, y_sat)
        self.light_model.fit(X_expanded, y_light)
    
    def generate_clothing_colors(self, skin_rgb, season, num_colors=8):
        """
        Generate clothing color recommendations based on skin RGB and season
        
        Parameters:
        -----------
        skin_rgb : tuple or list
            RGB values of the skin tone (0-255)
        season : str
            One of 'summer', 'autumn', 'winter', 'spring'
        num_colors : int
            Number of color recommendations to generate
            
        Returns:
        --------
        numpy.ndarray
            Array of recommended colors in RGB (0-255)
        """
        # Normalize RGB values
        r, g, b = [x / 255.0 for x in skin_rgb]
        
        # Convert to HSL for feature extraction
        h, l, s = colorsys.rgb_to_hls(r, g, b)
        
        # Create season one-hot encoding
        season_summer = 1 if season == 'summer' else 0
        season_autumn = 1 if season == 'autumn' else 0
        season_winter = 1 if season == 'winter' else 0
        season_spring = 1 if season == 'spring' else 0
        
        # Create feature vector
        X = np.array([[
            r, g, b, h, l, s,
            season_summer, season_autumn, season_winter, season_spring
        ]])
        
        # Scale features
        X_scaled = self.scaler.transform(X)
        
        # Make predictions for multiple colors by adding random noise to the input
        X_variations = np.vstack([
            X_scaled + np.random.normal(0, 0.1, X_scaled.shape) for _ in range(num_colors * 3)
        ])
        
        # Predict HSL values
        hue_preds = self.hue_model.predict(X_variations)
        sat_preds = self.sat_model.predict(X_variations)
        light_preds = self.light_model.predict(X_variations)
        
        # Convert predictions to valid HSL
        hue_preds = np.clip(hue_preds, 0, 1)
        sat_preds = np.clip(sat_preds, 0.1, 1)
        light_preds = np.clip(light_preds, 0.1, 0.9)
        
        # Convert back to RGB
        rgb_colors = []
        for h_pred, l_pred, s_pred in zip(hue_preds, light_preds, sat_preds):
            rgb = colorsys.hls_to_rgb(h_pred, l_pred, s_pred)
            rgb_colors.append([c * 255 for c in rgb])
        
        # Select the most diverse set of colors by clustering
        from sklearn.cluster import KMeans
        kmeans = KMeans(n_clusters=num_colors, random_state=42)
        kmeans.fit(rgb_colors)
        
        centers = kmeans.cluster_centers_
        
        # Return as numpy array
        return np.array(centers)
    
    def save_model(self, filename="color_model.joblib"):
        """Save the trained model to a file"""
        model_data = {
            'hue_model': self.hue_model,
            'sat_model': self.sat_model,
            'light_model': self.light_model,
            'scaler': self.scaler
        }
        joblib.dump(model_data, filename)
    
    @classmethod
    def load_model(cls, filename="color_model.joblib"):
        """Load a trained model from a file"""
        generator = cls()
        model_data = joblib.load(filename)
        generator.hue_model = model_data['hue_model']
        generator.sat_model = model_data['sat_model']
        generator.light_model = model_data['light_model']
        generator.scaler = model_data['scaler']
        return generator

In [4]:
def generate_clothing_colors(skin_rgb, season, num_colors=8):
    """
    Generate clothing color recommendations based on skin RGB and season
    
    Parameters:
    -----------
    skin_rgb : tuple or list
        RGB values of the skin tone (0-255)
    season : str
        One of 'summer', 'autumn', 'winter', 'spring'
    num_colors : int
        Number of color recommendations to generate
        
    Returns:
    --------
    numpy.ndarray
        Array of recommended colors in RGB (0-255)
    """
    generator = ClothingColorGenerator()
    return generator.generate_clothing_colors(skin_rgb, season, num_colors)

In [7]:
import pandas as pd
pd.read_csv('../csv/training_celebs.csv')

,name,season,undertone,Eye_R,Eye_G,Eye_B,Eye_Hex,Lips_R,Lips_G,Lips_B,Lips_Hex,Cheek_R,Cheek_G,Cheek_B,Cheek_Hex,Hair_R,Hair_G,Hair_B,Hair_Hex
0,Beyonce,Autumn,Warm,93,101,108,#5D656C,127,60,55,#7F3C37,138,78,60,#8A4E3C,77,50,41,#4D3229
1,Lady Gaga,Autumn,Warm,109,118,125,#6D767D,122,56,52,#7A3834,138,78,60,#8A4E3C,89,60,51,#593C33
2,Selena Gomez,Winter,Neutral,114,124,130,#727C82,84,31,26,#541F1A,140,80,61,#8C503D,85,56,46,#55382E
3,Hrithik Roshan,Autumn,Warm,112,120,126,#70787E,99,41,36,#632924,139,78,60,#8B4E3C,80,52,42,#50342A
4,Sarah Rafferty,Autumn,Neutral,110,119,125,#6E777D,118,54,49,#763631,140,80,62,#8C503E,83,55,45,#53372D
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Prince Harry,Spring,Warm,112,120,126,#70787E,87,33,28,#57211C,140,80,61,#8C503D,83,55,43,#53372B
96,Michael Peña,Autumn,Warm,97,104,111,#61686F,75,25,21,#4B1915,139,78,60,#8B4E3C,80,53,42,#50352A
97,Taylor Zakhar Perez,Winter,Cool,100,107,113,#646B71,114,51,48,#723330,140,80,61,#8C503D,83,55,46,#53372E
98,Maisie Richardson-Sellers,Winter,Cool,108,117,124,#6C757C,102,42,38,#662A26,139,79,60,#8B4F3C,77,50,41,#4D3229
